In [ ]:
# Try to append data to existing excel file, while checking first if file there try find file, 
# if found append excel method else create file
# Create info essentials, test new info def
# Check energy settings

In [1]:
from random import randint
from time import sleep
import datetime
import yfinance as yf
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [142]:
# Def for reading the ticker file as csv: csv_ticker('filename.csv', 'columnname')
def csv_ticker(csv_file, ticker_col):
    df = pd.read_csv(csv_file)
    df_tick = df[ticker_col].drop_duplicates(keep='first').dropna()
    return df_tick

# Def for reading the ticker file as xlsx: xlsx_ticker('filename.xlsx', 'columnname') returns "df_tick"
def xlsx_ticker(xlsx_file, ticker_col):
    df = pd.read_excel(xlsx_file)
    df_tick = df[ticker_col].drop_duplicates(keep='first').dropna()
    return df_tick

# Def for reading collected data after preprocessing through “def info()“ and “def fin()“
def csv_df(csv_file):
    df = pd.read_csv(csv_file)
    return df

def xlsx_df(xlsx_file):
    df = pd.read_excel(xlsx_file)
    return df

In [174]:
# Analysis constists of 3 tables: stock advisor company info = df_1, 
# SnP500 company info = df_2, stock tips = df_adv
df_1 = xlsx_df('test.info.xlsx')
df_2 = csv_df('df_info_465_2021.10.04 21_31_53.csv')
df_adv = csv_df('advisor stocks.csv')

# Company info tables are appended
df_1 = df_1.append(df_2)

# Duplicates are dropped to join with stock tips df_adv
df_1 = df_1.drop_duplicates(subset=['symbol'])
df_1.shape


# Stock tips df_adv has to be prepared for left join, renaming, drop duplicates

# rename key_column
df_adv = df_adv.rename(columns={'ticker': 'symbol', 'symbol':'symbol_'})

# drop duplicates with same symbol/ ticker
df_adv = df_adv.drop_duplicates(subset=['symbol'])
df_adv

# left join on 'symbol' getting columns 'status_advisor', 'date_advisor', 'price_when_advised', 'source_advisor'
df_1 = pd.merge(df_1, df_adv[['symbol','status_advisor','date_advisor',
                             'price_when_advised','source_advisor']],on='symbol', how='left')

(771, 171)

In [180]:
# Storing the result to prepare df for K-means
now = datetime.datetime.now()
now = now.strftime("%Y.%m.%d %H_%M_%S")
len_df_1 = str(len(df_1))
df_1.to_excel('df_1'+len_df+'_'+now+'.xlsx')

,Unnamed: 0,zip,sector,fullTimeEmployees,longBusinessSummary,city,phone,country,companyOfficers,website,...,postMarketPrice,exchangeName,preMarketChange,regularMarketSource,marketState,err,status_advisor,date_advisor,price_when_advised,source_advisor
0,0,1120,Technology,4538,"Kapsch TrafficCom AG provides technologies, so...",Vienna,43 50 811 0,Austria,[],http://www.kapsch.net,...,NaN,NaN,NaN,NaN,NaN,NaN,Kaufen,07.09.21,15.06,SA
1,0,74321,Industrials,17114,"DÃ¼rr Aktiengesellschaft, together with its su...",Bietigheim-Bissingen,49 7142 78 1785,Germany,[],http://www.durr-group.com,...,NaN,NaN,NaN,NaN,NaN,NaN,Kaufen,03.08.21,41.06,SA
2,0,90411,Technology,288,"Exasol AG, a technology company, develops data...",Nuremberg,49 911 23991 0,Germany,[],http://www.exasol.com,...,NaN,NaN,NaN,NaN,NaN,NaN,Kaufen,06.07.21,18.33,SA
3,0,56070,Healthcare,8030,CompuGroup Medical SE & Co. KGaA develops and ...,Koblenz,49 261 8000 0,Germany,[],http://www.cgm.com,...,NaN,NaN,NaN,NaN,NaN,NaN,Kaufchance,08.06.21,67.00,SA
4,0,81379,Technology,450,NFON AG provides cloud private branch exchange...,Munich,49 89 453 00 0,Germany,[],http://www.nfon.com,...,NaN,NaN,NaN,NaN,NaN,NaN,Kaufen,04.05.21,17.75,SA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
729,0,10573,Industrials,15600,"Xylem Inc., together with its subsidiaries, en...",Rye Brook,914 323 5700,United States,[],http://www.xylem.com,...,NaN,YHD,NaN,DELAYED,REGULAR,"{'elapsedTime': 261, 'headers': {'server': 'en...",NaN,NaN,NaN,NaN
730,0,40213,Consumer Cyclical,38000,"YUM! Brands, Inc., together with its subsidiar...",Louisville,502 874 8300,United States,[],http://www.yum.com,...,NaN,YHD,NaN,DELAYED,REGULAR,"{'elapsedTime': 261, 'headers': {'server': 'en...",NaN,NaN,NaN,NaN
731,0,46580,Healthcare,20000,"Zimmer Biomet Holdings, Inc., together with it...",Warsaw,574 267 6131,United States,[],http://www.zimmerbiomet.com,...,NaN,YHD,NaN,DELAYED,REGULAR,"{'elapsedTime': 261, 'headers': {'server': 'en...",NaN,NaN,NaN,NaN
732,0,84133-1109,Financial Services,9727,"Zions Bancorporation, National Association pro...",Salt Lake City,801 844 7637,United States,[],http://www.zionsbancorporation.com,...,NaN,YHD,NaN,DELAYED,REGULAR,"{'elapsedTime': 261, 'headers': {'server': 'en...",NaN,NaN,NaN,NaN


In [202]:
df_k = df_1[['sector', 'city', 'state', 'country', 'industry', 'revenueGrowth', 'debtToEquity', 
             'totalRevenue', 'financialCurrency', 'shortName', 'longName', 'symbol', 'priceToBook', 
             'payoutRatio', 'trailingAnnualDividendRate', 'trailingPE', 'marketCap',]]

In [204]:
df_k[['trailingAnnualDividendRate', 'payoutRatio']].fillna(0, inplace=True)

In [210]:
rg = df_k[df_k['revenueGrowth']!=0]
rg.mean()


/var/folders/5q/m0yt4w2n3279165l93qszgp00000gn/T/ipykernel_34805/2654980935.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  rg.mean()


revenueGrowth                 2.866423e+01
debtToEquity                  1.934841e+02
totalRevenue                  3.167781e+10
priceToBook                   2.231404e+01
payoutRatio                   3.984060e-01
trailingAnnualDividendRate    6.366780e+00
trailingPE                    9.232375e+01
marketCap                     6.194628e+10
dtype: float64

In [207]:
df_k.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 734 entries, 0 to 733
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   sector                      734 non-null    object 
 1   city                        734 non-null    object 
 2   state                       686 non-null    object 
 3   country                     734 non-null    object 
 4   industry                    734 non-null    object 
 5   revenueGrowth               730 non-null    float64
 6   debtToEquity                669 non-null    float64
 7   totalRevenue                733 non-null    float64
 8   financialCurrency           734 non-null    object 
 9   shortName                   728 non-null    object 
 10  longName                    725 non-null    object 
 11  symbol                      734 non-null    object 
 12  priceToBook                 690 non-null    float64
 13  payoutRatio                 696 non

In [116]:
# TESTING with samples
# splitting dataframe by row index to create test samples for data downloading
df_1 = df_tick.iloc[:5]
df_2 = df_tick.iloc[5:] # tested 20-50 (did not work). works until 46, 49-50 works fine 9-50 are missing now
df_3 = df_tick.iloc[51:100]
df_3 = df_tick.iloc[101:200]
df_4 = df_tick.iloc[201:]
print("Shape of new dataframes - {} , {} , {} , {}".format(df_1.shape, df_2.shape, df_3.shape, df_4.shape))

Shape of new dataframes - (5,) , (303,) , (99,) , (107,)


In [134]:
#Example tickers: ticker_list = ['BZ6.F',	'DUE.DE',	'EXL.DE',	'COP.DE',	'NFN.DE',	'MED.SW',
#                'SAP.DE',	'EVT.DE',	'AAG.DE',	'LPK.DE',	'OSP2.DE',	'ADN1.DE',	'SHF.DE',	
#               'SANT.DE',	'AOF.DE'] # example list
# Dict. ticker_info_dict has to be created to store information coming from .info function
# Within dict. values have to be iterable (lists, tuples, dict., sets) values returned from .info have to be 
# transformed into list. List of 308 ticker took circa 25 min.

def info(df):
    ticker_info_dict = {}
    ticker_info_dict2 = {}
    info_list = []
    total_number = 0
    total_passed = 0
    
    for tick in df:
        try:
            ticker_item = yf.Ticker(tick)
            ticker_info_dict = ticker_item.info
            print('Starting to loop through dict. of', tick)

            for key, value in ticker_info_dict.items():
                ticker_info_dict2[key] = [value]

            a = pd.DataFrame.from_dict(ticker_info_dict2)
            info_list.append(a)
               
            total_number = total_number + 1
            print('Nr: ' + str(total_number), tick)
        except Exception as e:
            total_passed = total_passed + 1
            print('Nr: ' + str(total_passed), tick, 'Passed!', e)
            #passed_df.append(tick, e)
            #passed_df.append([ticker, e]) # e text of exception, to understand reason why passed
            pass
    df_info = pd.concat(info_list, sort=False)  # transform info_list into df
    df_info.fillna(0)
    
    
    print('Start saving stock data...')

    #Create variables for specific naming of file, to not overwrite data
    #Later file shall be overwritten or appended for purpose of updating
    now = datetime.datetime.now()
    now = now.strftime("%Y.%m.%d %H_%M_%S")
    len_df = str(len(df))

    df_info.to_csv('df_info_'+len_df+'_'+now+'.csv')

    print('Stock data saved!')

In [139]:
def fin(df):
        balance_list = []
        cashflow_list = []
        financials_list = []
        total_number = 0
        total_passed = 0
        passed_df = []
        
        for tick in df:
            try:
                ticker = yf.Ticker(tick)
                a = ticker.balancesheet.T   # T stands for transpose
                if a.index.dtype.kind == 'M':  # Check if ticker is valid, if no data can be found then index.dtype
                                               # is an object and index is filled 
                                               # with ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
                    a.insert(0, 'ticker', tick)
                    b = ticker.cashflow.T
                    b.insert(0, 'ticker', tick)
                    c = ticker.financials.T
                    c.insert(0, 'ticker', tick)

                    balance_list.append(a)
                    cashflow_list.append(b)
                    financials_list.append(c)

                    total_number = total_number + 1
                    print('Nr: ' + str(total_number), tick)
                else:
                    total_passed = total_passed + 1
                    print('Nr: ' + str(total_passed), tick, 
                          'Passed! Ticker is empty or has no datetime index with index.dtype:', a.index.dtype)
            except Exception as e:
                total_passed = total_passed + 1
                print('Nr: ' + str(total_passed), tick, 'Passed!', e)
                #passed_df.append(tick, e)
                #passed_df.append([ticker, e]) # e text of exception, to understand reason why passed
                pass

        df_balance = pd.concat(balance_list, sort=False)
        df_balance.insert(1, 'Date', pd.DatetimeIndex(df_balance.index).date) # Add try except for date column issue
        df_balance.fillna(0)

        df_cashflow = pd.concat(cashflow_list, sort=False)
        df_cashflow.insert(1, 'Date', pd.DatetimeIndex(df_cashflow.index).date)
        df_cashflow.fillna(0)

        df_financials = pd.concat(financials_list, sort=False)
        df_financials.insert(1, 'Date', pd.DatetimeIndex(df_financials.index).date)
        df_financials.fillna(0)
        try:
            df_passed = pd.concat(passed_df, sort=False)
        except Exception as e:
            print(e)
            pass
        print('Start saving stock data...')
        #df_pass = pd.DataFrame(Screener.globalpass, index=None, columns={'Ticker', 'Error'})
        
        #Create variables for specific naming of the files, to not overwrite data
        #Later files shall be overwritten or appended for purpose of updating
        now = datetime.datetime.now()
        now = now.strftime("%Y.%m.%d %H_%M_%S")
        len_df = str(len(df))
        
        df_balance.to_csv('balance_'+len_df+'_'+now+'.csv')
        df_cashflow.to_csv('cashflow_'+len_df+'_'+now+'.csv')
        df_financials.to_csv('financials_'+len_df+'_'+now+'.csv')
        
        print('Stock data saved!')
        
        #Writer not needed, as analysis will be conducted with CSV format
        #writer = pd.ExcelWriter('merge.xlsx')
        #df_balance.to_excel(writer, 'Balances', index=False)
        #df_cashflow.to_excel(writer, 'Cashflows', index=False)
        #df_financials.to_excel(writer, 'Financials', index=False)
        #writer.save()

In [137]:
df_tick = xlsx_ticker('SnP500_open.xlsx', 'ticker')
df_tick

0       MMM
1       AOS
2       ABT
3      ABBV
4       ACN
       ... 
460     XYL
461     YUM
462     ZBH
463    ZION
464     ZTS
Name: ticker, Length: 465, dtype: object

In [138]:
info(df_tick)

Starting to loop through dict. of MMM
Nr: 1 MMM
Starting to loop through dict. of AOS
Nr: 2 AOS
Starting to loop through dict. of ABT
Nr: 3 ABT
Starting to loop through dict. of ABBV
Nr: 4 ABBV
Starting to loop through dict. of ACN
Nr: 5 ACN
Starting to loop through dict. of AYI
Nr: 6 AYI
Starting to loop through dict. of ADBE
Nr: 7 ADBE
Starting to loop through dict. of AAP
Nr: 8 AAP
Starting to loop through dict. of AMD
Nr: 9 AMD
Starting to loop through dict. of AES
Nr: 10 AES
Starting to loop through dict. of AET
Nr: 11 AET
Starting to loop through dict. of AMG
Nr: 12 AMG
Starting to loop through dict. of AFL
Nr: 13 AFL
Starting to loop through dict. of A
Nr: 14 A
Starting to loop through dict. of APD
Nr: 15 APD
Starting to loop through dict. of AKAM
Nr: 16 AKAM
Starting to loop through dict. of ALK
Nr: 17 ALK
Starting to loop through dict. of ALB
Nr: 18 ALB
Starting to loop through dict. of ARE
Nr: 19 ARE
Starting to loop through dict. of ALXN
Nr: 20 ALXN
Starting to loop through 

Starting to loop through dict. of EXR
Nr: 167 EXR
Starting to loop through dict. of XOM
Nr: 168 XOM
Starting to loop through dict. of FFIV
Nr: 169 FFIV
Starting to loop through dict. of FAST
Nr: 170 FAST
Starting to loop through dict. of FRT
Nr: 171 FRT
Starting to loop through dict. of FDX
Nr: 172 FDX
Starting to loop through dict. of FIS
Nr: 173 FIS
Starting to loop through dict. of FITB
Nr: 174 FITB
Starting to loop through dict. of FE
Nr: 175 FE
Starting to loop through dict. of FISV
Nr: 176 FISV
Starting to loop through dict. of FLIR
Nr: 177 FLIR
Starting to loop through dict. of FLS
Nr: 178 FLS
Starting to loop through dict. of FLR
Nr: 179 FLR
Starting to loop through dict. of FMC
Nr: 180 FMC
Starting to loop through dict. of FL
Nr: 181 FL
Starting to loop through dict. of F
Nr: 182 F
Starting to loop through dict. of FTV
Nr: 183 FTV
Starting to loop through dict. of FBHS
Nr: 184 FBHS
Starting to loop through dict. of BEN
Nr: 185 BEN
Starting to loop through dict. of FCX
Nr: 186 

Starting to loop through dict. of PRGO
Nr: 331 PRGO
Starting to loop through dict. of PFE
Nr: 332 PFE
Starting to loop through dict. of PCG
Nr: 333 PCG
Starting to loop through dict. of PM
Nr: 334 PM
Starting to loop through dict. of PSX
Nr: 335 PSX
Starting to loop through dict. of PNW
Nr: 336 PNW
Starting to loop through dict. of PXD
Nr: 337 PXD
Starting to loop through dict. of PNC
Nr: 338 PNC
Starting to loop through dict. of RL
Nr: 339 RL
Starting to loop through dict. of PPG
Nr: 340 PPG
Starting to loop through dict. of PPL
Nr: 341 PPL
Starting to loop through dict. of PX
Nr: 342 PX
Nr: 1 PCLN Passed! 'NoneType' object is not iterable
Starting to loop through dict. of PFG
Nr: 343 PFG
Starting to loop through dict. of PG
Nr: 344 PG
Starting to loop through dict. of PGR
Nr: 345 PGR
Starting to loop through dict. of PLD
Nr: 346 PLD
Starting to loop through dict. of PRU
Nr: 347 PRU
Starting to loop through dict. of PEG
Nr: 348 PEG
Nr: 2 PSA Passed! 'NoneType' object is not iterable
S

In [140]:
fin(df_tick)

Nr: 1 MMM
Nr: 2 AOS
Nr: 3 ABT
Nr: 4 ABBV
Nr: 5 ACN
Nr: 6 AYI
Nr: 7 ADBE
Nr: 8 AAP
Nr: 9 AMD
Nr: 10 AES
Nr: 1 AET Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 11 AMG
Nr: 12 AFL
Nr: 13 A
Nr: 14 APD
Nr: 15 AKAM
Nr: 16 ALK
Nr: 17 ALB
Nr: 18 ARE
Nr: 2 ALXN Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 19 ALLE
Nr: 3 AGN Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 20 ADS
Nr: 21 LNT
Nr: 22 ALL
Nr: 23 GOOGL
Nr: 24 MO
Nr: 25 AEE
Nr: 26 AAL
Nr: 27 AEP
Nr: 28 AXP
Nr: 29 AIG
Nr: 30 AWK
Nr: 31 AMP
Nr: 32 ABC
Nr: 33 AME
Nr: 34 AMGN
Nr: 35 APH
Nr: 4 APC Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 36 ADI
Nr: 5 ANDV Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 37 ANSS
Nr: 38 ANTM
Nr: 39 AON
Nr: 40 APA
Nr: 41 AIV
Nr: 42 AAPL
Nr: 43 AMAT
Nr: 44 APTV
Nr: 45 ADM
Nr: 46 ARNC
Nr: 47 AJG
Nr: 48 AIZ
Nr: 49 T
Nr: 50 ADP
Nr: 51 AZO
Nr: 52 AVB
Nr: 53 

Nr: 372 FOXA
Nr: 373 FOX
Nr: 374 TSN
Nr: 375 USB
Nr: 46 UDR Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 376 ULTA
Nr: 377 UAA
Nr: 378 UNP
Nr: 379 UAL
Nr: 380 UNH
Nr: 381 UPS
Nr: 382 URI
Nr: 47 UTX Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 383 UHS
Nr: 384 UNM
Nr: 385 VFC
Nr: 386 VLO
Nr: 48 VAR Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 387 VTR
Nr: 388 VRSN
Nr: 389 VRSK
Nr: 390 VZ
Nr: 49 VIAB Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 391 V
Nr: 392 VNO
Nr: 393 VMC
Nr: 394 WMT
Nr: 395 WBA
Nr: 396 WAT
Nr: 397 WEC
Nr: 398 WFC
Nr: 50 HCN Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 399 WDC
Nr: 400 WU
Nr: 401 WRK
Nr: 402 WY
Nr: 403 WHR
Nr: 404 WMB
Nr: 405 WLTW
Nr: 51 WYN Passed! Ticker is empty or has no datetime index with index.dtype: object
Nr: 406 WYNN
Nr: 407 XEL
Nr: 408 XRX
Nr: 409 XL
Nr: 410 XYL
Nr: 411 YUM
Nr: 412 Z

In [ ]:
def info_essentials(df)
    ticker_info_dict = {}
    ticker_info_dict2 = {}
    info_list = []
    total_number = 0
    total_passed = 0
    
    for tick in df:
        try:
            ticker_item = yf.Ticker(tick)
            ticker_info_dict = ticker_item.info # define here the few needed items to speed up the download
            print('Starting to loop through dict. of', tick)

            for key, value in ticker_info_dict.items():
                ticker_info_dict2[key] = [value]

            a = pd.DataFrame.from_dict(ticker_info_dict2)
            info_list.append(a)
               
            total_number = total_number + 1
            print('Nr: ' + str(total_number), tick)
        
        except Exception as e:
            total_passed = total_passed + 1
            print('Nr: ' + str(total_passed), tick, 'Passed!', e)
            #passed_df.append(tick, e)
            #passed_df.append([ticker, e]) # e text of exception, to understand reason why passed
            pass
        
    df_info = pd.concat(info_list, sort=False)  # transform info_list into df
    df_info.fillna(0)
    df_info.to_excel('test.info.xlsx')

In [ ]:
# Within dict. values have to be iterable (lists, tuples, dict., sets)
#df = pd.DataFrame(data=dic_msft)
dic2 = {}
for key, value in dic_msft.items():
    dic_msft[key] = [value]

In [ ]:
df_msft = pd.DataFrame.from_dict(dic_msft)

In [ ]:
df_msft

In [ ]:
df_msft.to_excel("msft.info.xlsx")

In [ ]:
# show income statement
dic_msft_fin={}
#dic_msft_fin = msft.financials
#msft.quarterly_financials

# show balance heet
dic_msft_bal={}
dic_msft_bal=msft.balance_sheet
msft.quarterly_balance_sheet

# show cashflow
dic_msft_cash={}
#dic_msft_cash=msft.cashflow
#msft.quarterly_cashflow

# show earnings
msft.earnings
msft.quarterly_earnings

# show sustainability
msft_profile=msft.sustainability

# show analysts recommendations
msft.recommendations

# show major holders
msft.major_holders

# show institutional holders
msft.institutional_holders

In [ ]:
# df['symbol']
# user input = 'AAPL'
# df.loc[df['symbol']==user input]
# if len(df.loc[df['symbol']==user input]) ==1
#    print('The following company was found:')
#    elif len(df.loc[df['symbol']==user input]) > 1
# df = df.loc
    

In [ ]:
# Selection idea
def user_selection():
    for index, item in enumerate(options):
                print(f"Option {index + 1} - {item}")
            while True:
                try: # Is used to avoid errors
                    choice = int(input("Selection: "))
                    if choice >= 1 and choice <= len(options):
                        return options[choice-1]
                except:
                    continue